In [1]:
from IPython.display import Image 
from PIL import Image
from torch.utils.data import Dataset
import albumentations as A 
from albumentations.pytorch  import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from matplotlib import pyplot as plt
import numpy as np
import os 
import re


In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
class DataFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        super(DataFolder,self).__init__() 
        self.data = []
        self.root_dir = root_dir
        self.transform = transform
        self.class_names= os.listdir(root_dir)
        # Loop through the folders and add labels to it signs:0 , notsigns: 1 
        for index, name in enumerate(self.class_names):
            if name == 'cctvsign':
                files = os.listdir(os.path.join(root_dir,name))
                self.data += list(zip(files, [index]*len(files)))
            else:
                files = os.listdir(os.path.join(root_dir,name))
                # Get the traffic images
                traffic = list(filter(lambda v: re.match(rf'^sign', v), files))
                #Get only 100 images per category of CIFAR 100 
                files = [list(filter(lambda v: re.match(rf'.*_{x}.jpg', v), files))[0:100] for x in range(20)]
                files = sum(files,[])+traffic
                self.data += list(zip(files, [index]*len(files)))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_file , label = self.data[index]
        root_and_dir = os.path.join(self.root_dir,self.class_names[label])
        image = np.array(Image.open(os.path.join(root_and_dir,img_file)).convert('RGB'))
        
        if self.transform is not None:
            augmentations= self.transform(image=image)
            image = augmentations['image']
            

        return image, label

train_transform = A.Compose(
    [
        A.Resize(32,32,always_apply=True),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.6),
        A.Rotate(limit=40,p=0.9),
        A.RGBShift(r_shift_limit=25, g_shift_limit=25,b_shift_limit=25,p=0.8),
        A.Normalize(
            mean=(0,0,0),
            std =(1,1,1),
            max_pixel_value=225,
        p=1),
        A.OneOf([
            A.Blur(blur_limit=3, p=0.5),
            A.ColorJitter(p=0.5)
        ],p=1.0),
        
        ToTensorV2(),
    ]
)
test_transform = A.Compose(
    [
        A.Resize(32,32,always_apply=True),
        A.Normalize(
            mean=(0,0,0),
            std =(1,1,1),
            max_pixel_value=225,
        p=1),
        
        ToTensorV2(),
    ]
)

In [9]:
#Load the datasets 
train_dataset = DataFolder(root_dir= r'dataset\train_data',transform=train_transform)
test_dataset = DataFolder(root_dir= r'dataset\validation_data',transform=test_transform)

batch_size = 64
trainloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


dataloaders = {'train': trainloader, 'test': testloader}
dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
def test_model(testloader, model):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = (correct/total)*100
    return accuracy

In [16]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [17]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.3892 Acc: 0.8418
test Loss: 0.2476 Acc: 0.9131

Epoch 1/24
----------
train Loss: 0.2885 Acc: 0.8885
test Loss: 0.1855 Acc: 0.9413

Epoch 2/24
----------
train Loss: 0.2413 Acc: 0.9079
test Loss: 0.1349 Acc: 0.9536

Epoch 3/24
----------
train Loss: 0.1897 Acc: 0.9317
test Loss: 0.1364 Acc: 0.9518

Epoch 4/24
----------
train Loss: 0.1898 Acc: 0.9299
test Loss: 0.2077 Acc: 0.9199

Epoch 5/24
----------
train Loss: 0.1607 Acc: 0.9356
test Loss: 0.1187 Acc: 0.9604

Epoch 6/24
----------
train Loss: 0.1723 Acc: 0.9403
test Loss: 0.1225 Acc: 0.9572

Epoch 7/24
----------
train Loss: 0.1372 Acc: 0.9504
test Loss: 0.1335 Acc: 0.9513

Epoch 8/24
----------
train Loss: 0.1411 Acc: 0.9417
test Loss: 0.1313 Acc: 0.9536

Epoch 9/24
----------
train Loss: 0.1382 Acc: 0.9450
test Loss: 0.1244 Acc: 0.9581

Epoch 10/24
----------
train Loss: 0.1378 Acc: 0.9468
test Loss: 0.1139 Acc: 0.9595

Epoch 11/24
----------
train Loss: 0.1335 Acc: 0.9446
test Loss: 0.1281 Acc

In [18]:
test_model(testloader, model_ft)

96.04005461993627